# 3. Load/Save Models and Accessing/Modifying Data

## 3.1. Objectives

**Purpose**

In this tutorial, you will learn to load and save models in [OpenSim](https://opensim.stanford.edu/) [[1]](https://doi.org/10.1109/TBME.2007.901024), as well as to access and modify the data in those models. This tutorial will lead you through the following:

*   Load an existing model.
*   Access and modify the information inside of a model.
*   Save your models.

**Format**

You will download a model from internet and load it into OpenSim. Then, you will access and print information from the model. Finally, you will modify the model and save and download it to your computer. 

**Resources**

All of the files necessary to complete this tutorial are available in the following Google Drive folder: https://drive.google.com/drive/folders/1VetbaL7jawe4w4bzYEjnCxQ6a1ZUMdk9?usp=sharing

While you complete the tutorial, you will find cells that automatically download the required files.

To obtain a link to one of the files in Google Drive, right-click over it and click on **Get Link**.

## 3.2. Setup Conda and OpenSim

First, set up the environment by executing the following cell (see [Tutorial 1: Introduction to OpenSim in Colab](https://drive.google.com/file/d/1P_2IRJFzdodS1-ce4BsOsC9d8xWXCqXS/view?usp=sharing) for more details).

Don't worry if after executing the previous cell you get an error saying that your session has failed. The reason for this is that condacolab needs to restart the session for the changes to take effect. Because of this, you have to execute the previous cell first, before executing any other cell.

Now, let's install the OpenSim conda package.

## 3.3. Load an existing model and explore the data

In this section you will download a model from the internet (from Google Drive), load it into OpenSim, and explore some of its properties. The model consists of a double pendulum.

First, let's download the model file (`double_pendulum.osim`). The file is stored in a Google Drive folder (See **Resources** in the Objectives section).

In [18]:
# !gdown "1Obs-_jEC0pLwnoMnATG8SXxCrUxZQWwx&confirm=t" # double_pendulum.osim

Now, let's import OpenSim, load the model, and print some metadata.

In [19]:
import opensim as osim

# Load the model.
double_pendulum = osim.Model('../resources/Tutorial 3/double_pendulum.osim')
double_pendulum.printToXML('double-pendulum-earth.osim')
# Print metadata.
print("Name of the model:", double_pendulum.getName())
print("Author:", double_pendulum.get_credits())
print("Publications:", double_pendulum.get_publications())
print("Length Unit:", double_pendulum.get_length_units())
print("Force Unit:", double_pendulum.get_force_units())
print("Gravity:", double_pendulum.get_gravity())

Name of the model: double_pendulum
Author: Ajay Seth
Publications: For testing
Length Unit: meters
Force Unit: N
Gravity: ~[0,-9.80665,0]


Let's explore some of the model's body properties. Since it is a double pendulum, we expect the model to have two bodies.

In [20]:
# Print the number of bodies.
print("Num Bodies:", double_pendulum.getNumBodies())
print()

# For each body, print some information, such as its name and mass.
for body in double_pendulum.getBodySet():
  print("Body Name:", body.getName())
  print("Body Absolute Path:", body.getAbsolutePathString())
  print("Body Mass:", body.getMass())
  print("Body Mass Center:", body.getMassCenter())
  print()

Num Bodies: 2

Body Name: rod1
Body Absolute Path: /bodyset/rod1
Body Mass: 1.0
Body Mass Center: ~[0,0.2,0]

Body Name: rod2
Body Absolute Path: /bodyset/rod2
Body Mass: 1.0
Body Mass Center: ~[0,0,0]



Let's explore some of the model's joint properties. Again, we expect to have two joints. The first one from the ground to rod1, so it is fixed and doesn't fall when you simulate it; and the second one from rod1 to rod2, linking the two bodies of the double pendulum.

In [21]:
# Print the number of joints.
print("Num Joints:", double_pendulum.getNumJoints())
print()

# For each joint, print some information, such as its name or components that
# this joint owns.
for joint in double_pendulum.getJointSet():
  print("Joint Name:", joint.getName())
  print("Joint Absolute Path:", joint.getAbsolutePathString())
  print("Components:")
  for component in joint.getComponentsList():
    print("  Component Name:", component.getName())
    print("  Component Absolute Path:", component.getAbsolutePathString())
  print()

Num Joints: 2

Joint Name: pin1
Joint Absolute Path: /jointset/pin1
Components:
  Component Name: q1
  Component Absolute Path: /jointset/pin1/q1
  Component Name: ground_offset
  Component Absolute Path: /jointset/pin1/ground_offset
  Component Name: frame_geometry
  Component Absolute Path: /jointset/pin1/ground_offset/frame_geometry
  Component Name: wrapobjectset
  Component Absolute Path: /jointset/pin1/ground_offset/wrapobjectset
  Component Name: rod1_offset
  Component Absolute Path: /jointset/pin1/rod1_offset
  Component Name: frame_geometry
  Component Absolute Path: /jointset/pin1/rod1_offset/frame_geometry
  Component Name: wrapobjectset
  Component Absolute Path: /jointset/pin1/rod1_offset/wrapobjectset

Joint Name: pin2
Joint Absolute Path: /jointset/pin2
Components:
  Component Name: q2
  Component Absolute Path: /jointset/pin2/q2
  Component Name: rod1_offset
  Component Absolute Path: /jointset/pin2/rod1_offset
  Component Name: frame_geometry
  Component Absolute Path

Finally, let's explore the coordinates of the model.

In [22]:
# Print the number of coordinates.
print("Num Coordinates:", double_pendulum.getNumCoordinates())
print()

# For each coordinate, print some information, such as its name or motion type.
for coordinate in double_pendulum.getCoordinateSet():
  print("  Coordinate Name:", coordinate.getName())
  print("  Coordinate Absolute Path:", coordinate.getAbsolutePathString())
  
  # Motion type is an enumerate (0:Undefined, 1:Rotational, 2:Translational, 3:Coupled).
  motion_type = coordinate.getMotionType()
  motion_type_string = ""
  if motion_type == 0:
    motion_type_string = "Undefined"
  elif motion_type == 1:
    motion_type_string = "Rotational"
  elif motion_type == 2:
    motion_type_string = "Translational "
  elif motion_type == 3:
    motion_type_string = "Coupled "
  print("  Coordinate Motion Type:", motion_type_string)
  
  print()

Num Coordinates: 2

  Coordinate Name: q1
  Coordinate Absolute Path: /jointset/pin1/q1
  Coordinate Motion Type: Rotational

  Coordinate Name: q2
  Coordinate Absolute Path: /jointset/pin2/q2
  Coordinate Motion Type: Rotational



## 3.4. Modify the model and save it

Let's modify some of the model's properties. For example, let's set the model's gravity to be gravity on the planet Mars (3.72 $m/s^2$), and double the mass of the bodies. After that, we want to save the changes, so we will save the model into a new file.

First, let's modify the gravity of the model to mars' gravity (3.72 $m/s^2$), and let's double the mass of the bodies.

In [23]:
import numpy as np

# Set gravity of mars and print it.
double_pendulum.set_gravity(osim.Vec3(0, -3.72, 0))
print("New Gravity:", double_pendulum.get_gravity())
print()

# For each body, print its name, set the mass as 2Kg, and print the updated mass.
for body in double_pendulum.getBodyList():
  print("Body Name:", body.getName())
  body.setMass(2.0)
  print("Body New Mass:", body.getMass())
  print()

New Gravity: ~[0,-3.72,0]

Body Name: rod1
Body New Mass: 2.0

Body Name: rod2
Body New Mass: 2.0



Now, let's save the model into an `osim` file.

In [24]:
double_pendulum.printToXML('heavy-double-pendulum-mars.osim');

The file can be downloaded in the Files pane onthe left by `right-clicking` on it and clicking `download` (if the file is not in the tree at the left, you may need to `right-click` and `refresh`).

Alternatively, you can download the file programmatically as shown in the following cell.

In [25]:
# from google.colab import files

# files.download('heavy-double-pendulum-mars.osim') 

## 3.5. Conclusion

In this tutorial, you have downloaded a double pendulum model and loaded it into OpenSim. Then you have explored some of its properties and components, and finally you have modified it and saved it into an `.osim` file, that you downloaded to your computer.

## 3.6. Useful Links





> **OpenSim Website:** https://opensim.stanford.edu/
>
> **OpenSim API Documentation:** https://simtk.org/api_docs/opensim/api_docs/
> 
> **OpenSim Creator Website:** https://opensimcreator.com/
> 
> **SimTK Website:** https://simtk.org/projects/opensim
> 
> **Biomechanics of Movement Course Videos:** https://www.youtube.com/channel/UCDNGy0KKNLQ-ztcL5h2Z6zA

##3.7 Acknowledgments

Thanks to [OpenSimColab](https://simtk.org/projects/opencolab) project [[2]](https://doi.org/10.1080/10255842.2022.2104607) for creating the first OpenSim Conda package.

## 3.8. References




> [1].   Delp, S. L., Anderson, F. C., Arnold, A. S., Loan, P., Habib, A., John, C. T., Guendelman, E., & Thelen, D. G. (2007). **OpenSim: open-source software to create and analyze dynamic simulations of movement.** *IEEE Transactions on Bio-Medical Engineering*, 54(11), 1940–1950. https://doi.org/10.1109/TBME.2007.901024
>
> [2] Mokhtarzadeh, H., Jiang, F., Zhao, S., & Malekipour, F. (2022). **OpenColab project: OpenSim in Google colaboratory to explore biomechanics on the web.** *Computer Methods in Biomechanics and Biomedical Engineering*, 1–9. https://doi.org/10.1080/10255842.2022.2104607